# IMDB Movie Analysis using AutoML


In [1]:
# import dependancies
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from sklearn.preprocessing import LabelEncoder 
import spacy
from collections import Counter   # for getting freq of words
from collections import OrderedDict
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory

In [2]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="imdb-automl")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-142761
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-142761


In [3]:
# Choose a name for your cluster.
compute_cluster_name = "capstone-automl"
vm_size="STANDARD_NC6"
max_nodes=8
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Cluster found with the name {0}.. Proceed'.format(compute_cluster_name))
except ComputeTargetException:
    print('Creating a new cluster with the name {0}'.format(compute_cluster_name))    
    compute_config = AmlCompute.provisioning_configuration(vm_size = vm_size, max_nodes = max_nodes)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating a new cluster with the name capstone-automl
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
ds=TabularDatasetFactory.from_delimited_files(path="https://raw.githubusercontent.com/Varsh18/IMDB-Moview-Review-Analysis/master/IMDB-Dataset.csv")


In [5]:
df=ds.to_pandas_dataframe()
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
#Convert text labels to number 
le = LabelEncoder() 
df['sentiment']= le.fit_transform(df['sentiment']) 
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# Preprocessing data

In [7]:
df['review']=df['review'].apply(lambda x: x.lower())

In [8]:
#function to remove stopwords
en_model = spacy.load('en_core_web_sm')
stopwords = en_model.Defaults.stop_words
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [9]:
def removeTags(data):
  cleanhtmltags = re.compile('<.*?>')
  return re.sub(cleanhtmltags, '', data)
df['review'] = df['review'].apply(removeTags)

In [10]:
##Expand contractions

# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['review']=df['review'].apply(lambda x:expand_contractions(x))


In [11]:
#remove punctuations & other special characters
df['review'] = df['review'].str.replace("[^a-zA-Z]", " ")

In [12]:
# remove the characters of length < 2 which doesn't add value
df['review'] = df['review'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

In [13]:
review_string = " ".join(review for review in df["review"])
review_string[0:1000]

'reviewers mentioned watching episode you will hooked right exactly happened the thing struck brutality unflinching scenes violence set right word trust faint hearted timid pulls punches regards drugs sex violence hardcore classic use word called nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda city home many aryans muslims gangstas latinos christians italians irish more scuffles death stares dodgy dealings shady agreements far away main appeal fact goes shows would not dare forget pretty pictures painted mainstream audiences forget charm forget romance does not mess around episode saw struck nasty surreal could not ready watched more developed taste got accustomed high levels graphic violence violence injustice crooked guards who will sold nickel inmates who will kill order away mannered middle class inmates turned prison bitches lack street skills prison experience watc

In [14]:
review_words = review_string.split(' ')

# get the frequency of each word (token) in the tweet string
review_words_freq = Counter(review_words)

# get the most frequent words
most_common_words = review_words_freq.most_common(400)

# view the most common words
most_common_words

[('movie', 87937),
 ('film', 79676),
 ('not', 63302),
 ('like', 40160),
 ('good', 29737),
 ('the', 28919),
 ('time', 25099),
 ('story', 23095),
 ('bad', 18461),
 ('people', 18182),
 ('great', 18137),
 ('way', 15644),
 ('movies', 15301),
 ('characters', 14444),
 ('think', 14335),
 ('watch', 13942),
 ('character', 13900),
 ('films', 13752),
 ('that', 13710),
 ('you', 13530),
 ('seen', 13370),
 ('love', 13006),
 ('plot', 12976),
 ('life', 12913),
 ('acting', 12858),
 ('best', 12611),
 ('know', 12508),
 ('this', 12437),
 ('little', 12430),
 ('man', 11824),
 ('there', 11426),
 ('better', 11426),
 ('end', 11115),
 ('scene', 10957),
 ('are', 10855),
 ('and', 10731),
 ('scenes', 10472),
 ('did', 10324),
 ('does', 9683),
 ('well', 9568),
 ('real', 9426),
 ('thing', 9173),
 ('watching', 9160),
 ('actors', 8948),
 ('director', 8804),
 ('years', 8760),
 ('funny', 8755),
 ('old', 8650),
 ('one', 8602),
 ('work', 8544),
 ('actually', 8466),
 ('all', 8358),
 ('makes', 8307),
 ('look', 8293),
 ('find'

In [15]:
common_word_list =[]
for i in range(len(most_common_words)):
  common_word_list.append(most_common_words[i][0])

In [16]:
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word in (common_word_list)]))

In [17]:
df['review'] = df['review'].str.split().apply(lambda x: ' '.join(OrderedDict.fromkeys(x).keys()))

In [18]:
df1=df.reindex(columns = ['sentiment','review'])
df1.head()

,sentiment,review
0,1,watching episode you will right the thing scen...
1,1,wonderful little production the old time gives...
2,1,thought wonderful way time watching comedy plo...
3,0,there family little boy time this movie kill y...
4,1,love time money film watch human movie people ...


In [19]:
x=df.iloc[:,:-1]
x

,review
0,watching episode you will right the thing scen...
1,wonderful little production the old time gives...
2,thought wonderful way time watching comedy plo...
3,there family little boy time this movie kill y...
4,love time money film watch human movie people ...
...,...
49995,thought movie right good job was not original ...
49996,bad plot dialogue acting script end the girl p...
49997,high school good eyes not believe things act m...
49998,going one second trying time like kid film dir...


In [20]:
y=df1.iloc[:,:-1]
y

,sentiment
0,1
1,1
2,1
3,0
4,1
...,...
49995,1
49996,0
49997,0
49998,0


In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
# split dataset into train and test sets
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size= 0.3, random_state = 0)
label = 'sentiment'

In [22]:
x_train

,review
17967,version face life father film men women the pl...
32391,the performance however things classic man wom...
9341,world war the set acting makes this moment nic...
7929,saw movie voice took big beginning humor looks...
46544,funny bit kind beginning what little pretty go...
...,...
21243,set high movie left story little overall think...
45891,great horror movie title character real feel g...
42613,years book the house finally bit which especia...
43567,fan thought series best comedy seen different ...


In [23]:
y_train

,sentiment
17967,1
32391,0
9341,1
7929,1
46544,1
...,...
21243,1
45891,1
42613,1
43567,0


In [24]:
x_train = pd.DataFrame(x_train)
x_train.columns = ['review']
x_train.head()

,review
17967,version face life father film men women the pl...
32391,the performance however things classic man wom...
9341,world war the set acting makes this moment nic...
7929,saw movie voice took big beginning humor looks...
46544,funny bit kind beginning what little pretty go...


In [25]:
y_train = pd.DataFrame(y_train)
y_train.columns = ['sentiment']
y_train.head()

,sentiment
17967,1
32391,0
9341,1
7929,1
46544,1


In [26]:
# merge the output x and y dataframes into a single table for AutoML experiment
train_data_df = pd.concat([x_train, y_train],axis=1)
# view train dataset
train_data_df.head()

,review,sentiment
17967,version face life father film men women the pl...,1
32391,the performance however things classic man wom...,0
9341,world war the set acting makes this moment nic...,1
7929,saw movie voice took big beginning humor looks...,1
46544,funny bit kind beginning what little pretty go...,1


In [27]:
# save training data in tabular format to allow for remote run
if not os.path.isdir('imbd-data'):  # create data folder if it does not exist
    os.mkdir('imbd-data')
    
# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("imbd-data/train_data.csv", index=False)


In [28]:
# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./imbd-data', target_path='imbd-capstone', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('imbd-capstone/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./imbd-data/train_data.csv
Uploaded ./imbd-data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [29]:
# experiment name
experiment_name = 'imdb-automl'
project_folder = './imbd-capstone'

In [30]:
# Set parameters for AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
# Configure Automl settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "primary_metric": 'accuracy',
    "max_concurrent_iterations": max_nodes, 
    "max_cores_per_iteration": -1,
    "enable_dnn": True,
    "enable_early_stopping": True,
    "validation_size": 0.3,
    "verbosity": logging.INFO,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_cluster_name,
                             training_data=train_data,
                             label_column_name='sentiment',
                             blocked_models = ['LightGBM', 'XGBoostClassifier'],
                             **automl_settings
                            )


SDK version: 1.26.0


In [31]:
automl_run = exp.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on capstone-automl with default configuration
Running on remote compute: capstone-automl


Experiment,Id,Type,Status,Details Page,Docs Page
imdb-automl,AutoML_1c07a841-cd90-4cc6-8c98-5b5708422cdf,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************

In [33]:
# show run details
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [34]:
# wait for completion
automl_run.wait_for_completion()

{'runId': 'AutoML_1c07a841-cd90-4cc6-8c98-5b5708422cdf',
 'target': 'capstone-automl',
 'status': 'Completed',
 'startTimeUtc': '2021-04-15T12:27:25.827262Z',
 'endTimeUtc': '2021-04-15T13:02:35.919813Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"imdb-automl","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-142761","workspace_name":"quick-starts-ws-142761","region":"southcentralus","compute_target":"capstone-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validat

In [36]:
# Retrieve the best automl run model
best_automl_run, fitted_automl_model = automl_run.get_output()
print('Best AutoML run: ', best_automl_run)
print('Best AutoML model :', fitted_automl_model)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print('Best_model name: ', model_name)

Best AutoML run:  Run(Experiment: imdb-automl,
Id: AutoML_1c07a841-cd90-4cc6-8c98-5b5708422cdf_20,
Type: azureml.scriptrun,
Status: Completed)
Best AutoML model : Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('StandardScalerWrapper',
                 <azureml...red.model_wrappers.StandardScalerWrapper object at 0x7f8c05831278>),
                ('LogisticRegression',
                 LogisticRegression(C=1.7575106248547894, class_weight=None,
                           

In [37]:
# display all the properties of the best model
best_automl_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': 'a11e2a30b92abbc8a5f30448aa77078bd421a6e7',
 'pipeline_spec': '{"objects": [{"class_name": "StandardScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"with_mean": false, "with_std": false}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "LogisticRegression", "module": "sklearn.linear_model", "param_args": [], "param_kwargs": {"C": 1.7575106248547894, "class_weight": null, "multi_class": "ovr", "penalty": "l2", "solver": "saga"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "a11e2a30b92abbc8a5f30448aa77078bd421a6e7", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipeline_name": "{ StandardScaler, LogisticRegression }"}',
 'training_percent': '100',
 'predicted_cost': '3.198926621701898',
 'iteration': '20',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetType': 'amlcompute',
 'ContentSnapshotId': '61578159-81a1-4a84-95af-ed62148ea11

In [38]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
imdb-automl,AutoML_1c07a841-cd90-4cc6-8c98-5b5708422cdf_20,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [39]:
#TODO: Save the best AutoML model
import joblib

joblib.dump(fitted_automl_model, 'best_automl_model.pkl')

['best_automl_model.pkl']

In [40]:
automl_model = automl_run.register_model(model_name=best_automl_run.properties['model_name'], description='Best AutoML model')

In [41]:
# Download scoring file 
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'automlscore.py')

# Download environment file
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'automlenvFile.yml')
best_automl_run.download_file('outputs/env_dependencies.json', 'automlenvDependencies.json')

In [42]:
from azureml.core.webservice import AciWebservice
from azureml.core import Model # Used to get model information

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name='project_environment',file_path='automlenvFile.yml')

inference_config = InferenceConfig(entry_script='automlscore.py',
                                    environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights=True)
  
service = Model.deploy(ws, "automl-imdb-sentiment-final", [automl_model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-15 13:42:10+00:00 Creating Container Registry if not exists.
2021-04-15 13:42:11+00:00 Building image..
2021-04-15 13:54:50+00:00 Generating deployment configuration.
2021-04-15 13:54:52+00:00 Submitting deployment to compute..
2021-04-15 13:55:00+00:00 Checking the status of deployment automl-imdb-sentiment-final..
2021-04-15 13:59:01+00:00 Checking the status of inference endpoint automl-imdb-sentiment-final.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://f35a3d9f-ede3-40a1-8150-093a86b0cad9.southcentralus.azurecontainer.io/score
http://f35a3d9f-ede3-40a1-8150-093a86b0cad9.southcentralus.azurecontainer.io/swagger.json


In [43]:
print(service.get_logs())

2021-04-15T13:58:48,504975400+00:00 - rsyslog/run 
2021-04-15T13:58:48,516599000+00:00 - iot-server/run 
2021-04-15T13:58:48,515966000+00:00 - gunicorn/run 
2021-04-15T13:58:48,573950700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [44]:
best_automl_run.properties['model_name']

'AutoML1c07a841c20'

In [45]:
Model.get_model_path(best_automl_run.properties['model_name'],_workspace=ws)

'azureml-models/AutoML1c07a841c20/1/model.pkl'

In [46]:
import json
#Creating Test data
#Created test data from the original data bytaking first 10 rows
test_df = df.sample(10) 
test_df_label = test_df.pop('sentiment')

test_data = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_data)

{"data": [{"review": "like long classic film characters you are looking book instead excellent think probably beautiful man world brilliant actor and course simply best actress"}, {"review": "totally kill great show seen episode big time lost start tell actually acting parts pretty good stuff obviously watching given does not real says you know guy better read book"}, {"review": "son found time idea liked way told movie recommend one actors known also lives people did not feel that definitely heart goes"}, {"review": "horror film best show comedy beautiful time people young long love hand extremely father way out town comes girl played coming like actor called boy turns john performance actors story dark fun original scenes play flick movie goes character funny humor line great movies one entertaining"}, {"review": "plays war leave love human this film excellent nice time laugh though there title long come again"}, {"review": "script acting story little would not bad she director write

In [48]:
# We will send POST request to the deployed service for that we will import the request module
import requests 
headers = {'Content-type': 'application/json'}
response = requests.post(service.scoring_uri, test_data, headers=headers)

In [49]:
response.text

'"{\\"result\\": [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]}"'

In [50]:
test_df_label.to_list()

[1, 0, 1, 1, 1, 0, 0, 1, 0, 1]

In [51]:
service.delete()

In [53]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

